In [1]:
using BenchmarkTools
using Plots

include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

In [3]:
cj.mass_from_radius(1/(0.028), cj.SharpK)

1.0658909692837272e17

In [ ]:
using Plots

Mhost_init = 1e+12
Mres = 1e+0

s_vs_m, ds_vs_m = cj.interpolation_s_vs_mass()

Mhost = 10.0.^range(log10(2.1*Mres), log10(Mhost_init), 200)

#function_P, function_F = cj.interpolate_functions_PF(Mhost_init, Mres)

plot(Mhost, cj.mean_number_progenitors.(Mhost/2.0, Mhost, Mres, s_vs_m, ds_vs_m), xscale=:log10,  linestyle=:solid, yscale=:log10)
#plot!(Mhost, function_P.(Mhost, Mres), xscale=:log10, linestyle=:dash, linewidth=4, yscale=:log10)

In [ ]:
using Plots

mhost = 1e+12
mres = 1e+2

m = 10.0.^range(log10(mres), log10(mhost), 200)

s_vs_m, _ = cj.interpolation_s_vs_mass()
#function_P, function_F = cj.interpolate_functions_PF(mhost, mres)

println(cj.mass_fraction_unresolved.(2e+2, 1e+2, s_vs_m))

plot(m, cj.mass_fraction_unresolved.(m, mres, s_vs_m), xscale=:log10,  linestyle=:solid)
#plot!(m, function_F.(m), xscale=:log10, linestyle=:dash, linewidth=4)

In [ ]:
using Plots

M1   = 1e+12
M2 = 10.0.^range(2, log10(M1), 200)

s_vs_m, ds_vs_m = cj.interpolation_s_vs_mass()

plot(M2[M2.>1e+8], cj.pdf_progenitors.(M2[M2.>1e+8], M1, 1e+8, s_vs_m, ds_vs_m), xscale=:log10, yscale=:log10)
plot!(M2[M2.>1e+5], cj.pdf_progenitors.(M2[M2.>1e+5], M1, 1e+5, s_vs_m, ds_vs_m), xscale=:log10, yscale=:log10)
plot!(M2[M2.>1e+2], cj.pdf_progenitors.(M2[M2.>1e+2], M1, 1e+2, s_vs_m, ds_vs_m), xscale=:log10, yscale=:log10)

In [ ]:
using Plots


s_vs_m, ds_vs_m = cj.interpolation_s_vs_mass()

M1   = 1e+12
M2 = 10.0.^range(0, log10(M1), 100)

#plot(M2, cj.cmf_progenitors.(M2, M1, 1e+8) .- 0.2, xscale=:log10)
#plot!(M2, cj.cmf_progenitors.(M2, M1, 1e+5).- 0.2, xscale=:log10)
#plot!(M2, cj.cmf_progenitors.(M2, M1, 1e+2).- 0.2, xscale=:log10)

M1   = 1e+12
M2 = 10.0.^range(1.5, log10(M1), 1000)

#plot(M2, cj.cmf_progenitors.(M2, M1, 1e+8).- 0.2, xscale=:log10, linestyle=:dash)
#plot!(M2, cj.cmf_progenitors.(M2, M1, 1e+5).- 0.2, xscale=:log10, linestyle=:dash)
plot(M2, cj.cmf_progenitors.(M2, M1, 1e+2, s_vs_m, ds_vs_m), xscale=:log10, linestyle=:dash)

In [ ]:
itp_z_vs_Δω = cj.interpolate_functions_z()

In [ ]:
subhalo_mass1, m_host1, z_steps1, z_acc1 = cj.subhalo_mass_function(1e+12, 1e+8, z_vs_Δω = itp_z_vs_Δω)

In [ ]:
subhalo_mass2, m_host2, z_steps2, z_acc2 = cj.subhalo_mass_function(1e+12, 1e+5, z_vs_Δω = itp_z_vs_Δω)

In [ ]:
subhalo_mass3, m_host3, z_steps3, z_acc3 = cj.subhalo_mass_function(1e+12, 1e+6,  z_vs_Δω = itp_z_vs_Δω)

In [ ]:
using Plots

m_array = 10.0.^range(-8, stop=1, length=1000)
#CMF1 = [count(x -> x > m, subhalo_mass1) for m in m_array]
CMF2 = [count(x -> x > m, subhalo_mass2) for m in m_array]
CMF3 = [count(x -> x > m, subhalo_mass3) for m in m_array]

#plot(m_array[CMF1 .> 0], CMF1[CMF1 .> 0], xscale=:log10, yscale=:log10, color=:blue)
plot(m_array[CMF2 .> 0], CMF2[CMF2 .> 0], xscale=:log10, yscale=:log10, color=:red)
plot!(m_array[CMF3 .> 0], CMF3[CMF3 .> 0], xscale=:log10, yscale=:log10, color=:green)

In [ ]:
using SpecialFunctions
using LsqFit

function γ2(γ1::Real, α1::Real, α2::Real, β::Real, ζ::Real) 
    int1 =  β.^(.-(2.0.-α1)./ζ) .* gamma.((2.0.-α1)./ζ) .*  gamma_inc.((2.0.-α1)./ζ, β)[1] ./ ζ 
    int2 =  β.^(.-(2.0.-α2)./ζ) .* gamma.((2.0.-α2)./ζ) .*  gamma_inc.((2.0.-α2)./ζ, β)[1] ./ ζ 
    return (1.0 .- γ1 .* int1) ./  int2
end

function fitting_function(m_m0, γ1, α1, α2, β, ζ)
    if α1 > 2 || α1 < 1 || α2 > 2 || α2 < 1 || β < 0 || ζ < 0 || γ1 < 0
        return Inf
    end
    return γ1./ζ .* ((m_m0).^(1.0.-α1) .* expint.((α1.-1.0)./ζ .+ 1.0, β.*(m_m0).^ζ) .- expint.((α1.-1.0)./ζ .+ 1.0, β)) .+  γ2(γ1, α1, α2, β, ζ)./ζ .* ((m_m0).^(1.0.-α2) .* expint.((α2.-1.0)./ζ .+ 1.0, β.*(m_m0).^ζ) .- expint.((α2.-1.0)./ζ .+ 1.0, β)) 
end

p0 = [0.019, 1.94, 1.58, 24, 3.4]

model(t, p) = @. log10.(fitting_function(10.0.^t, p[1], p[2], p[3], p[4], p[5]))

fit = curve_fit(model, log10.(m_array[CMF3 .> 0]), log10.(CMF3[CMF3 .> 0]), p0)

println(fit.param)
println(γ2(fit.param...))

res = 10.0.^model(log10.(m_array), fit.param) #fitting_function(1e+12 .* m_array, 1e+12, fit.param...)
plot(m_array[res .> 0], res[res .> 0], xscale=:log10, yscale=:log10, ylim=[1e-1, 1e+5])
plot!(m_array[CMF3 .> 0], CMF3[CMF3 .> 0], xscale=:log10, yscale=:log10, ylim=[1e-1, 1e+5])

In [ ]:
#plot(z_steps1, m_host1, yscale=:log10, color=:blue)
plot(z_steps2, m_host2, yscale=:log10, color=:red)
plot!(z_steps3, m_host3, yscale=:log10, color=:green)

In [ ]:
#plot(subhalo_mass1, z_acc1, seriestype=:scatter, xscale=:log10, markersize = 2, yscale=:log10)
plot(subhalo_mass2, z_acc2, seriestype=:scatter, xscale=:log10, markersize = 2, yscale=:log10)
plot!(subhalo_mass3, z_acc3, seriestype=:scatter, xscale=:log10, markersize = 2, yscale=:log10)

In [ ]:
import StatsBase as SB
using Plots

id_6 = (subhalo_mass2 .> 1e-6) .& (subhalo_mass2 .< 1e-5)
id_5 = (subhalo_mass2 .> 1e-5) .& (subhalo_mass2 .< 1e-4)
id_4 = (subhalo_mass2 .> 1e-4) .& (subhalo_mass2 .< 1e-3)
id_3 = (subhalo_mass2 .> 1e-3) .& (subhalo_mass2 .< 1e-2)
#println(count(x -> x === true, id_4))

#result6 = SB.fit(Histogram, log10.(z_acc3[id_6]), nbins=40)
#result5 = SB.fit(Histogram, log10.(z_acc3[id_5]), nbins=40)
#plot(result6.edges[1][2:end], log10.(result6.weights), st=:stairs)
#plot!(result5.edges[1][2:end], log10.(result5.weights), st=:stairs)

histogram(log10.(z_acc2[id_6]), normalize=:pdf)
histogram!(log10.(z_acc2[id_5]), normalize=:pdf)
histogram!(log10.(z_acc2[id_4]), normalize=:pdf)

In [ ]:
include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

mres = 1e+3
mhost = 1e+12

#cj.save_data_merger_tree(mhost, mres)

In [ ]:
using Plots

q = range(-10, 0, length=101)
m1_array = 10.0.^range(log10(2.0001*mres), log10(mhost), length=101)

func, _, _ = cj.load_data_merger_tree(mhost, mres)
plot(heatmap(q, log10.(m1_array), log10.(func.((1.0 .- 10.0.^q)', m1_array)), color = :turbo))
contour!(q, log10.(m1_array), log10.(func.((1.0 .- 10.0.^q)', m1_array)), color=:black, levels=16)

In [ ]:
using Plots

p = range(0, 0.999, length=501)
m1_array = 10.0.^range(log10(2.0001*mres), log10(mhost), length=501)

func, _, _ = cj.load_data_merger_tree(mhost, mres)
#plot(heatmap(p, log10.(m1_array), log10.(func.(p', m1_array))), c = :thermal)
contourf(p, log10.(m1_array), log10.(func.(p', m1_array)), color=:turbo)
plot!([0, 1], [5, 5])

In [ ]:
include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

using Random

Random.seed!(3)

itp_z_vs_Δω = cj.interpolate_functions_z()

z_bins, mass_edges, z_edges = cj.subhalo_mass_function_binned(1e+12, 1e+3, z_vs_Δω = itp_z_vs_Δω)

In [ ]:
using Plots

mass_bins = sum(z_bins, dims=2)[:]
edges = mass_edges[1:end-1][mass_bins .> 0]

plot(edges,  log(10) .* mass_bins[mass_bins .> 0], xscale=:log10, yscale=:log10, seriestype=:stairs)

In [ ]:
using Plots

im = searchsortedfirst(mass_edges, 1e-7)

#mass_edges[21]

edges = z_edges[1:end-1]
data1 = z_bins[im,:]
data2 = z_bins[im-1,:]
data3 = z_bins[im+1,:]

plot(edges[data1 .> 0], data1[data1 .> 0], xscale=:log10, yscale=:log10, seriestype=:stairs)
plot!(edges[data2 .> 0], data2[data2 .> 0], xscale=:log10, yscale=:log10, seriestype=:stairs)
plot!(edges[data3 .> 0], data3[data3 .> 0], xscale=:log10, yscale=:identity, seriestype=:stairs)

In [ ]:
include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

m1_array = 10.0.^range(log10(1e+6), log10(mhost), length=101)
res1 = cj.cmf_inv_progenitors.(0.9999999999, m1_array, 1e+2, itp_S_vs_mass, itp_dS_vs_mass)
res2 = cj.cmf_inv_progenitors.(0.999999999, m1_array, 1e+2, itp_S_vs_mass, itp_dS_vs_mass)
res3 = cj.cmf_inv_progenitors.(0.99999999, m1_array, 1e+2, itp_S_vs_mass, itp_dS_vs_mass)
res4 = cj.cmf_inv_progenitors.(0.9999999, m1_array, 1e+2, itp_S_vs_mass, itp_dS_vs_mass)
res5 = cj.cmf_inv_progenitors.(0.999999, m1_array, 1e+2, itp_S_vs_mass, itp_dS_vs_mass)

plot(m1_array[res1 .> 0], res1[res1 .> 0], xscale=:log10, yscale=:log10, legend = false, color=:blue)
plot!(m1_array[res2 .> 0], res2[res2 .> 0], xscale=:log10, yscale=:log10, legend = false, color=:red)
plot!(m1_array[res3 .> 0], res3[res3 .> 0], xscale=:log10, yscale=:log10, legend = false, color=:green)
plot!(m1_array[res4 .> 0], res4[res4 .> 0], xscale=:log10, yscale=:log10, legend = false, color=:purple)
plot!(m1_array[res5 .> 0], res5[res5 .> 0], xscale=:log10, yscale=:log10, legend = false, color=:orange)

In [ ]:
m_range = 10.0.^range(log10(1e+4), 12, 200)

s_vs_m, dS_vs_m = cj.interpolation_s_vs_mass()
plot(m_range, cj.cmf_progenitors.(1e+5, m_range, 1e+3, s_vs_m, dS_vs_m ), xscale=:log10)



In [ ]:
s_vs_m, dS_vs_m = cj.interpolation_s_vs_mass()

m_range = 10.0.^range(log10(2.001e+5), 14, 200)

plot(m_range, cj.cmf_inv_progenitors.(0.5, m_range, 1e+5, s_vs_m, dS_vs_m), xscale=:log10)

In [ ]:
s_vs_m, dS_vs_m = cj.interpolation_s_vs_mass()

m_range = 10.0.^range(log10(2.1e+3), 12, 200)

plot(m_range, cj.cmf_inv_progenitors.(0.5, m_range, 1e+3, s_vs_m, dS_vs_m), xscale=:log10)
#println(cj.cmf_inv_progenitors.(0.5, 1e+5, 1e+3, s_vs_m, dS_vs_m))
#println(func(0.12112736878992725, 2.4e+3))

In [ ]:
include("../src/Cosmojuly.jl")
import .Cosmojuly as cj


custom_power_spectrum(k_Mpc::Real) = 1e-5/sqrt(2.0*π*0.5^2)/k_Mpc*exp(-(log(k_Mpc/10.0))^2/2/0.5^2) + cj.power_spectrum_ΛCDM(k_Mpc, 1e-10*exp(3.044), 0.9649)

k_range = 10.0.^range(-2, 5, 100)

plot(k_range, custom_power_spectrum.(k_range), xscale=:log10, yscale=:log10)

In [ ]:
new_cosmology = cj.Cosmology("planck_with_spike", cj.planck18_bkg, custom_power_spectrum, cj.EH98_planck18)


In [ ]:
include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

mres = 1e+1
mhost = 1e+12

custom_power_spectrum(k_Mpc::Real) = 1e-5/sqrt(2.0*π*0.5^2)/k_Mpc*exp(-(log(k_Mpc/10.0))^2/2/0.5^2) + cj.power_spectrum_ΛCDM(k_Mpc, 1e-10*exp(3.044), 0.9649)
new_cosmology = cj.Cosmology("planck_with_spike", cj.planck18_bkg, custom_power_spectrum, cj.EH98_planck18)

#cj.save_data_merger_tree(mhost, mres, cosmology=new_cosmology)

In [ ]:
using Plots

q = range(-10, 0, length=201)
m1_array = 10.0.^range(log10(2.0001*mres), log10(mhost), length=201)

func, _, _ = cj.load_data_merger_tree(mhost, mres, cosmology=new_cosmology)
#plot(heatmap(q, log10.(m1_array), log10.(func.((1.0 .- 10.0.^q)', m1_array)), color = :turbo))
contour!(q, log10.(m1_array), log10.(func.((1.0 .- 10.0.^q)', m1_array)), color=:black, levels=16)

In [ ]:
using Plots

p = range(0, 0.999, length=501)
m1_array = 10.0.^range(log10(2.0001*mres), log10(mhost), length=501)

func, _, _ = cj.load_data_merger_tree(mhost, mres, cosmology=new_cosmology)
#plot(heatmap(p, log10.(m1_array), log10.(func.(p', m1_array))), c = :thermal)
contourf(p, log10.(m1_array), log10.(func.(p', m1_array)), color=:turbo)

In [ ]:
using Random

Random.seed!(0)

itp_z_vs_Δω = cj.interpolate_functions_z()
z_bins_new, mass_edges_new, z_edges_new = cj.subhalo_mass_function_binned(1e+12, 1e+1, z_vs_Δω = itp_z_vs_Δω, cosmology=new_cosmology)
z_bins, mass_edges, z_edges = cj.subhalo_mass_function_binned(1e+12, 1e+3, z_vs_Δω = itp_z_vs_Δω)

In [ ]:
using Plots
using LaTeXStrings

mass_bins = sum(z_bins, dims=2)[:]
Δm = diff(mass_edges)
edges = mass_edges[1:end-1]
mask = mass_bins .> 0

mass_bins_new = sum(z_bins_new, dims=2)[:]
Δm_new = diff(mass_edges_new)
edges_new = mass_edges_new[1:end-1]
mask_new = mass_bins_new .> 0


plot(edges[mask],  edges[mask].^2 .* mass_bins[mask] ./ Δm[mask], seriestype=:stairs, label=L"\textrm{Planck18}")
plot!(edges_new[mask_new],  edges_new[mask_new].^2 .*  mass_bins_new[mask_new] ./ Δm_new[mask_new], seriestype=:stairs, label=L"\textrm{Planck18 + spike}")
xlabel!(L"m/m_{\textrm{host}}", xguidefontsize=14)
ylabel!(L"m^2\frac{\textrm{d} N}{{\textrm{d}} m} ~~ \textrm{[M_\odot]}", xguidefontsize=14)
xticks!([1e-10, 1e-8, 1e-6, 1e-4, 1e-2, 1])
plot!(framestyle=:box, legend=:topleft, xscale=:log10, yscale=:log10,  size=(500, 400))
savefig("../figs/subhalo_mass_function_with_spike.pdf")

In [ ]:
using Plots
using LaTeXStrings

m_range = 10.0.^range(-12, 1, 1000)

mass_bins = sum(z_bins, dims=2)[:]
edges = mass_edges[1:end-1]
cmf = [convert.(Int, sum(mass_bins[edges .> m])) for m in m_range]

mass_bins_new = sum(z_bins_new, dims=2)[:]
edges_new = mass_edges_new[1:end-1]
cmf_new = [convert.(Int, sum(mass_bins_new[edges_new .> m])) for m in m_range]

plot(m_range[cmf .> 0],  cmf[cmf .> 0], seriestype=:stairs, label=L"\textrm{Planck18}")
plot!(m_range[cmf_new .> 0],  cmf_new[cmf_new .> 0], seriestype=:stairs, label=L"\textrm{Planck18 + spike}")
xlabel!(L"m/m_{\textrm{host}}", xguidefontsize=14)
ylabel!(L"m^2\frac{\textrm{d} N}{{\textrm{d}} m} ~~ \textrm{[M_\odot]}", xguidefontsize=14)
xticks!([1e-10, 1e-8, 1e-6, 1e-4, 1e-2, 1])
plot!(framestyle=:box, legend=:topright, xscale=:log10, yscale=:log10,  size=(500, 400))
savefig("../figs/subhalo_cmf_with_spike.pdf")

In [ ]:
using Plots

#im = searchsortedfirst(mass_edges_new, 1e-6)

edges = z_edges_new[1:end-1]
data = z_bins_new[40,:]

plot(edges[data .> 0], data[data .> 0], xscale=:log10, seriestype=:stairs)

In [ ]:
include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

custom_power_spectrum(k_Mpc::Real) = 1e-5/sqrt(2.0*π*0.5^2)/k_Mpc*exp(-(log(k_Mpc/10.0))^2/2/0.5^2) + cj.power_spectrum_ΛCDM(k_Mpc, 1e-10*exp(3.044), 0.9649)
new_cosmology = cj.Cosmology("planck_with_spike", cj.planck18_bkg, custom_power_spectrum, cj.EH98_planck18)


m_range = 10.0.^range(1, 12, 10000)
s_vs_m_here, ds_vs_m = cj.interpolation_s_vs_mass(new_cosmology, range=range(1, 12, 10000))
signal =  cj.σ²_vs_M.(m_range, cj.SharpK, cosmology=new_cosmology)
approx = s_vs_m_here.(m_range)
error = 2.0.*abs.(signal.-approx)./(approx .+ signal)
plot(m_range, error, xscale=:log10)

#plot!(m_range, cj.σ²_vs_M.(m_range, cj.SharpK, cosmology=new_cosmology), xscale=:log10)

In [ ]:
plot(m_range, log10.(cj.σ²_vs_M.(m_range, cj.SharpK, cosmology=new_cosmology)), xscale=:log10)

In [ ]:
cj.σ²_vs_M(369.62111745609815, cj.SharpK, cosmology=new_cosmology) #8394.394587008283 1030.0663414599542
#cj.σ²_vs_M(1030.0663414599542, cj.SharpK, cosmology=new_cosmology)

In [ ]:
s_vs_m_here(369.62111745609815)
#s_vs_m(1030.0663414599542)